In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

In [2]:
# File to Load (Remember to Change These)
data_file = os.path.join("Resources", "purchase_data.csv")

In [3]:
# Read Purchasing File and store into Pandas data frame
data_file_df = pd.read_csv(data_file, encoding="utf-8")

In [4]:
data_file_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#Verify that there aren't missing entries
data_file_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [6]:
#Find the total number of unique players, create a data frame for the information, print out data frame
total_players = len(data_file_df["SN"].unique())
total_players_df = pd.DataFrame({"Total Players": [total_players]})
total_players_df

,Total Players
0,576


In [59]:
#For formatting money
def money (x):
    return x.map("${:.2f}".format)

In [56]:
#For formatting percentages
def percent (x):
    return x.map('{:,.2%}'.format)

In [9]:
#Calculate the values needed for the "Purchasing Analysis (Total)" data frame
unique_items = len((data_file_df["Item Name"].unique()))
ave_price = data_file_df["Price"].mean()
num_purchases = len(data_file_df)
total_revenue = data_file_df["Price"].sum()

In [20]:
#Create and print out the purchase summary as a data frame
purchase_summ_df = pd.DataFrame({"Unique Items": [unique_items],
                                 "Average Price": "${:,.2f}".format(ave_price),
                                 "Number of Purchases": num_purchases,
                                 "Total Revenue": "${:,.2f}".format(total_revenue)})

purchase_summ_df

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [11]:
#Create dataframe for unique player info (SN, Age, Gender)
player_info_df = data_file_df.drop_duplicates(subset = "SN")
player_info_df = player_info_df.drop(["Purchase ID", "Item ID", "Item Name", "Price"], axis=1)

In [121]:
#Create dataframe for the analysis by gender, using unique player info only
gender_total = player_info_df["Gender"].value_counts()  #total number of unique players by gender (M + F + O)
gender_percent = (gender_total / total_players).map('{:,.2%}'.format)

gender_df = pd.DataFrame({"Total Count": gender_total,
                          "Percentage of Players": gender_percent})

gender_df.index.name = "Gender"

gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [125]:
# DF with pertinent gender and purchase information
gender_data_df = data_file_df[["Gender", "Price"]]

#Calculate Values for Purchase Count, using all purchase info
purchase_gender_count = gender_data_df["Gender"].value_counts()

#Create index and first column of gender analysis data frame, initialize dataframe here to cut down on variable names
gender_summ_df = pd.DataFrame({"Purchase Count": purchase_gender_count})
gender_summ_df.index.name = "Gender"

# Create the column for "Average Purchase Price"
gender_summ_df["Average Purchase Price"] = gender_data_df.groupby(["Gender"]).mean()
gender_summ_df["Average Purchase Price"] = gender_summ_df["Average Purchase Price"].map("${:.2f}".format)

#Create the column for "Total Purchase Value"
gender_summ_df["Total Purchase Value"] = gender_data_df.groupby(["Gender"]).sum()
gender_summ_df["Total Purchase Value"] = gender_summ_df["Total Purchase Value"].map("${:.2f}".format)

#Create the column for Avg Purchase Total per Person by Gender (Total Purchase Value / Total Players)
gender_summ_df["Avg Total Purchase per Person"] = gender_data_df.groupby(["Gender"]).sum().divide(total_players)
gender_summ_df["Avg Total Purchase per Person"] = gender_summ_df["Avg Total Purchase per Person"].map("${:.2f}".format)

#Display Purchasing Analysis (Gender)
gender_summ_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Male,652,$3.02,$1967.64,$3.42
Female,113,$3.20,$361.94,$0.63
Other / Non-Disclosed,15,$3.35,$50.19,$0.09


In [25]:
#Create age bins and labels
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
age_display = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
age_bins_unique = (pd.cut(player_info_df["Age"], age_bins, labels = age_display, include_lowest = True))

#Calculate the total players in each group, and the percent of the total (using unique player info)
age_total = age_bins_unique.value_counts()
age_percent = (age_total/total_players).map('{:,.2%}'.format)

#Create dataframe and sort it by the age ranges
age_df = pd.DataFrame({"Total Count": age_total,
                          "Percentage of Players": age_percent})

age_df.index.name = "Age Ranges"
age_df = age_df.sort_values(by = ["Age Ranges"])

#Display the Age Demographics data frame
age_df


,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [37]:
# DF with pertinent gender and purchase information
age_data_df = data_file_df[["Age", "Price"]]

#Binned data, using all info
age_bins_all = pd.cut(age_data_df["Age"], age_bins, labels = age_display, include_lowest = True)
#age_grouped = age_data_df.groupby("Age Ranges")
#age_data_binned_df = age_data_df["Age Ranges"]

#Calculate Values for Purchase Count
purchase_age_count = age_bins_all.value_counts()

#Create index and first column of age analysis data frame, initialize dataframe here to cut down on variable names
age_summ_df = pd.DataFrame({"Purchase Count": purchase_age_count})

# Create the column for "Average Purchase Price"
age_summ_df["Average Purchase Price"] = age_data_df.groupby(age_bins_all)["Age"].mean()
age_summ_df["Average Purchase Price"] = age_summ_df["Average Purchase Price"].map("${:.2f}".format)

#Create the column for "Total Purchase Value"
age_summ_df["Total Purchase Value"] = age_data_df.groupby(age_bins_all)["Age"].sum().map("${:.2f}".format)
#age_summ_df["Total Purchase Value"] = age_summ_df["Total Purchase Value"].map("${:.2f}".format)

#Create the column for Avg Purchase Total per Person by Gender (Total Purchase Value / Total Players)
age_summ_df["Avg Total Purchase per Person"] = age_data_df.groupby(age_bins_all)["Age"].sum().divide(total_players)
age_summ_df["Avg Total Purchase per Person"] = age_summ_df["Avg Total Purchase per Person"].map("${:.2f}".format)

age_summ_df.index.name = "Age Ranges"

#Display Purchasing Analysis (Gender)
age_summ_df = age_summ_df.sort_values(by = ["Age Ranges"])
age_summ_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$7.87,$181.00,$0.31
10-14,28,$11.39,$319.00,$0.55
15-19,136,$16.79,$2284.00,$3.97
20-24,365,$21.84,$7971.00,$13.84
25-29,101,$26.00,$2626.00,$4.56
30-34,73,$31.38,$2291.00,$3.98
35-39,41,$36.71,$1505.00,$2.61
40+,13,$41.54,$540.00,$0.94


In [16]:
spenders_df = data_file_df[["SN", "Price"]]
spenders_df.head()

,SN,Price
0,Lisim78,3.53
1,Lisovynya38,1.56
2,Ithergue48,4.88
3,Chamassasya86,3.27
4,Iskosia90,1.44


In [138]:
spenders_df = data_file_df[["SN", "Price"]]
sn_spenders_df = spenders_df.groupby(["SN"])

top_spenders_df = sn_spenders_df.sum()
top_spenders_df = top_spenders_df.sort_values(["Price"], ascending = False)


top_spenders_df["Average Purchase Price"] = sn_spenders.mean().apply(money)
top_spenders_df["Purchase Count"] = data_file_df["SN"].value_counts()

top_spenders_df

,Price,Average Purchase Price,Purchase Count
SN,,,
Lisosia93,18.96,$3.79,5
Idastidru52,15.45,$3.86,4
Chamjask73,13.83,$4.61,3
Iral74,13.62,$3.40,4
Iskadarya95,13.10,$4.37,3
...,...,...,...
Ililsasya43,1.02,$1.02,1
Irilis75,1.02,$1.02,1
Aidai61,1.01,$1.01,1


In [74]:
popular_df = data_file_df[["Item ID", "Item Name", "Price"]]
pro

,Item ID,Item Name,Price
0,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,143,Frenzied Scimitar,1.56
2,92,Final Critic,4.88
3,100,Blindscythe,3.27
4,131,Fury,1.44


In [98]:
items_df = popular_df.groupby("Item Name")

top_popular = items_df.value_counts()
#top_popular = top_popular.sort_values(["Price"], ascending = False)

items_df

AttributeError: 'DataFrameGroupBy' object has no attribute 'groupby'